In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from utils import load_train_data, load_test_data

In [3]:
data_dir = "../reduced_data/"
#data_dir = "../data/"
batch_size = 32
img_height = 224
img_width = 224

In [4]:
train_data, train_labels = load_train_data(data_dir)
test_data, test_labels = load_test_data(data_dir)

Found 293 files belonging to 29 classes.


2022-02-23 22:00:00.275448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/franblue/anaconda3/envs/XAIFL/lib/python3.7/site-packages/cv2/../../lib64:
2022-02-23 22:00:00.275484: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-23 22:00:00.275514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (frandd): /proc/driver/nvidia/version does not exist
2022-02-23 22:00:00.276412: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 10 files belonging to 1 classes.


In [33]:
train_labels = tf.one_hot(train_labels, 29)

In [35]:
resnet = tf.keras.applications.resnet.ResNet101(include_top = False, weights='imagenet', input_shape=(224, 224, 3))
resnet.trainable = False
classifier = resnet.output
classifier = tf.keras.layers.GlobalAveragePooling2D()(classifier)
classifier = tf.keras.layers.Dense(29, activation='softmax')(classifier)
model = tf.keras.Model(inputs=resnet.input, outputs=classifier)

In [36]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
metrics = [tf.keras.metrics.categorical_accuracy]
patience = 10
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=0, mode='auto', restore_best_weights=True)]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [37]:
model.fit(train_data, train_labels, epochs=1, callbacks=callbacks, batch_size=2)

2022-02-23 22:38:07.491665: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 176418816 exceeds 10% of free system memory.


147/147 [==============================] - 100s 610ms/step - loss: 3.4880 - categorical_accuracy: 0.0239
